In [ ]:
from main import main
from glob import glob
import os


In [ ]:
# Global variables
OPTIMIZER = None
SHOTS = None
X_DIM = None
BACKEND = None
DEVICE = None
SCALE_FACTORS = None
ANSATZ = None
ENCODER = None
POSTPROCESS = None
ERROR_MITIGATION = None
LAYERS = None
PROVIDER = None
TOKEN = None
HYPERPARAMETERS = None
RE_UPLOAD_DEPTH = None
MAX_ITER = None
TOLERANCE = None
NUM_QUBITS = None

In [ ]:
topdir='/home/grierjones/qregress'
settings=sorted(glob('function-calc-test/lin5qubits/*/*json'))

# settings="./function-calc-test/5qubits/A2-A2-CNOT_ESU2/A2-A2-CNOT_ESU2.json"
train_set="./function-calc-test/linear/linear_train.bin"
test_set="./function-calc-test/linear/linear_test.bin"
scaler="./function-calc-test/linear/linear_scaler.bin"

In [ ]:
for i in settings:
    # print(os.path.dirname(i),i)
    namedir=os.path.dirname(i)
    done=namedir+'.done'
    if os.path.exists(done)==False:
        print(done)
        os.chdir(namedir)
        main(i, train_set, test_set, scaler)
        print(f'finished {namedir}')